PROJECT-3. Решение комплексной бизнес-задачи
Студент Солодухина Е. П.

 ЗАДАЧИ (ФОРМАЛИЗОВАННЫЕ):
 
    1. Построить рекомендательную систему, благодаря которой можно будет предлагать клиентам интересные им курсы. Для этого потребуется подготовить и проанализировать имеющиеся данные.
    2. Составить итоговую таблицу с рекомендациями, снабдив её необходимыми комментариями, и представить  продакт-менеджеру.
  

Познакомимся с датасетом и подготовим данные при помощи SQL
 
select 
	c.table_schema,
	c.table_name,
	c.column_name,
	c.data_type,
	c.ordinal_position
from 
	information_schema.columns c
where 
	c.table_schema = 'final'
    
    Продажи за какие годы есть в наших данных?
                
Select distinct(extract (year from c.purchased_at))
from 
    final.carts c
join final.cart_items ci
    on c.id=ci.cart_id
where
    c.state='successful' and ci.resource_type='Course'
    
Результат: 2018, 2017
                
     Сколько клиентов покупали курсы?

Select count(distinct(c.user_id)) 
   from final.carts c
   join final.cart_items ci
   on c.id=ci.cart_id
   where c.state='successful' and ci.resource_type='Course'
   
Результат: 49006
    
    Сколько всего есть различных курсов?
    
Select count(distinct ci.resource_id)
   from final.cart_items as ci
   where ci.resource_type='Course'
   
Результат: 127
   
    Каково среднее число купленных курсов на одного клиента?
    
Select round(avg(f.res),2) from
(Select count (ci.resource_id)/count(distinct c.user_id) res
   from final.carts c
   inner join final.cart_items ci
   on c.id=ci.cart_id
   where c.state='successful' and ci.resource_type='Course'
   group by c.user_id)f
   
Результат: 1,44
   
    Сколько клиентов купили больше одного курса?
    
Select sum(f.id) from
(select count(distinct c.user_id) id
from final.carts c
inner join final.cart_items ci
   on c.id=ci.cart_id
where c.state='successful' and ci.resource_type='Course'
group by c.user_id
having count(distinct ci.resource_id)>1) f 

Результат: 12656

    Сформируем данные по продажам курсов в разрезе пользователей (по купившим больше одного курса)
    
with user_tab as (
   select c.user_id, count(ci.resource_id)
   from final.carts c
   inner join final.cart_items ci
   on c.id=ci.cart_id
   where c.state='successful' and ci.resource_type='Course'
   group by c.user_id
   having count(distinct ci.resource_id)>1
   )
   select c.user_id, ci.resource_id
   from final.carts c
   inner join final.cart_items ci
       on c.id=ci.cart_id
   inner join user_tab u
       on c.user_id=u.user_id
   where c.state='successful' and ci.resource_type='Course'
   
    Итак, полученные данные сохраняем в файл, далее перейдём к его анализу с помощью Python.

In [1]:
#Импортируем необходимые библиотеки, получаем данные из файлов
import pandas as pd 
import numpy as np
from itertools import combinations
from collections import Counter

In [2]:
final_df=pd.read_csv('final.csv')
final_df.head()

,user_id,resource_id
0,1010882,490
1,906674,357
2,160494,507
3,749529,489
4,1010802,514


In [3]:
final_df=final_df.drop_duplicates()
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34074 entries, 0 to 34171
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   user_id      34074 non-null  int64
 1   resource_id  34074 non-null  int64
dtypes: int64(2)
memory usage: 798.6 KB


In [4]:
#Произведём группировку, чтобы каждому пользователю соответствовал список всех купленных им курсов
df_grouped=final_df.groupby(by='user_id')['resource_id'].apply(lambda x:list(x)).reset_index()
df_grouped

,user_id,resource_id
0,51,"[516, 1099]"
1,6117,"[1125, 356, 357]"
2,10275,"[553, 1147]"
3,10457,"[361, 1138]"
4,17166,"[356, 357]"
...,...,...
12651,2179430,"[566, 750]"
12652,2186581,"[794, 1129, 864]"
12653,2187601,"[356, 912, 571, 765, 553]"
12654,2188926,"[743, 515]"


In [5]:
#Сколько различных пар курсов встречаются вместе в покупках клиентов?
courses_pairs =[]
for element in df_grouped['resource_id']:
    for pair in combinations(sorted(list(element)),2):
        courses_pairs.append(pair)
#Удаляем дубликаты
courses_pairs
print('Количество различных пар курсов составляет: ', len(set (courses_pairs)))

Количество различных пар курсов составляет:  3989


In [6]:
courses_pairs

[(516, 1099),
 (356, 357),
 (356, 1125),
 (357, 1125),
 (553, 1147),
 (361, 1138),
 (356, 357),
 (1125, 1140),
 (551, 745),
 (553, 745),
 (551, 1138),
 (553, 568),
 (514, 517),
 (514, 566),
 (517, 566),
 (363, 511),
 (363, 562),
 (363, 563),
 (511, 562),
 (511, 563),
 (562, 563),
 (568, 745),
 (509, 553),
 (509, 745),
 (553, 745),
 (1125, 1144),
 (509, 568),
 (509, 672),
 (568, 672),
 (516, 552),
 (356, 552),
 (357, 571),
 (568, 745),
 (509, 516),
 (509, 568),
 (516, 568),
 (513, 1141),
 (571, 1125),
 (551, 552),
 (551, 744),
 (551, 862),
 (551, 1138),
 (552, 744),
 (552, 862),
 (552, 1138),
 (744, 862),
 (744, 1138),
 (862, 1138),
 (361, 1138),
 (356, 679),
 (571, 745),
 (571, 1099),
 (745, 1099),
 (509, 516),
 (509, 568),
 (509, 1099),
 (516, 568),
 (516, 1099),
 (568, 1099),
 (517, 750),
 (800, 1125),
 (569, 840),
 (509, 568),
 (745, 1125),
 (509, 514),
 (509, 551),
 (509, 745),
 (514, 551),
 (514, 745),
 (551, 745),
 (571, 765),
 (745, 1187),
 (363, 566),
 (513, 1125),
 (1100, 1103

In [7]:
popular=Counter([pair for pair in courses_pairs])
#Какая самая популярная пара курсов?
{k: v for k, v in popular.items() if v == max(popular.values())}

{(551, 566): 797}

In [8]:
df_cp = pd.DataFrame(courses_pairs,  columns=['cource_1', 'course_2']).astype(str)
df_cp.drop_duplicates()
df_cp['pair'] = df_cp['cource_1'] + "," + df_cp['course_2']
df_cp.describe(include='all')
df_cp                                                                            

,cource_1,course_2,pair
0,516,1099,"516,1099"
1,356,357,"356,357"
2,356,1125,"356,1125"
3,357,1125,"357,1125"
4,553,1147,"553,1147"
...,...,...,...
40012,765,912,"765,912"
40013,515,743,"515,743"
40014,756,794,"756,794"
40015,756,1185,"756,1185"


In [9]:
grouped_cp=df_cp.groupby(by='pair').count()
grouped_cp=grouped_cp.reset_index()
grouped_cp=grouped_cp[['pair', 'cource_1']].rename(columns={'cource_1':'count'})
grouped_cp

,pair,count
0,"1099,1100",5
1,"1099,1101",2
2,"1099,1102",2
3,"1099,1103",4
4,"1099,1125",4
...,...,...
3984,"912,1144",2
3985,"912,1156",1
3986,"912,1186",9
3987,"912,1187",1


In [10]:
#Оставим для анализа пары курсов, поввторяющиеся большее количество раз, для этого зададим 90 процентиль 
x=np.percentile(grouped_cp['count'], 90)
x

22.0

In [11]:
popular_pairs1= {k: v for k, v in sorted(popular.items(), key = lambda item: item[1], reverse = True) if v>x}
popular_pairs1

{(551, 566): 797,
 (515, 551): 417,
 (489, 551): 311,
 (523, 551): 304,
 (566, 794): 290,
 (489, 515): 286,
 (490, 566): 253,
 (490, 551): 247,
 (570, 752): 247,
 (569, 572): 216,
 (515, 523): 213,
 (553, 745): 212,
 (489, 523): 206,
 (569, 840): 204,
 (514, 551): 200,
 (516, 745): 199,
 (515, 566): 195,
 (489, 566): 188,
 (504, 572): 184,
 (572, 840): 178,
 (551, 552): 177,
 (507, 570): 172,
 (490, 809): 163,
 (489, 490): 152,
 (507, 752): 150,
 (523, 552): 144,
 (490, 515): 143,
 (551, 570): 142,
 (504, 569): 139,
 (514, 515): 139,
 (551, 745): 138,
 (514, 566): 138,
 (502, 551): 135,
 (504, 840): 135,
 (571, 1125): 122,
 (502, 566): 120,
 (523, 566): 120,
 (570, 809): 119,
 (752, 809): 115,
 (490, 523): 114,
 (357, 571): 112,
 (523, 564): 110,
 (551, 749): 109,
 (516, 553): 107,
 (551, 777): 107,
 (551, 679): 104,
 (356, 571): 103,
 (551, 564): 103,
 (515, 749): 103,
 (568, 745): 102,
 (356, 357): 100,
 (363, 511): 99,
 (551, 571): 98,
 (551, 809): 96,
 (502, 514): 95,
 (551, 794): 

In [12]:
# Создадим функцию recommend, которая будет рекомендовать 1-ый и 2-ой по популярности курс к основному курсу:
def recommend(course):
    course_list = []
    for i in popular_pairs1.keys():
        if i[0] == course:
            course_list.append((i, popular_pairs1[i]))
    sorted_course_list = sorted(course_list, key = lambda x: x[1], reverse = True)
    return sorted_course_list[:2] #выводим только два значения

In [13]:
course = final_df['resource_id'].unique()

In [14]:
#Заведём переменную с парой самых популярных курсов для вывода значений по умолчанию вместо Nan рекомендаций
s={k: v for k, v in popular.items() if v == max(popular_pairs1.values())}
for d in s.keys():
    print(d)

(551, 566)


In [15]:
# Создадим цикл, который для каждого основного курса извлекает 2-ое значение пары согласно уникальным значениям курсов:
#По умолчанию вместо Nan значений используем данные самых продаваемых курсов
recommended_courses1 = pd.DataFrame(columns = ['Рекомендация № 1', 'Рекомендация № 2'])
for i in course:
    if len(recommend(i)) == 2: #когда функция выдаёт 2 рекомендации
        recommended_courses1.loc[i] = [recommend(i)[0][0][1], recommend(i)[1][0][1]] 
    elif len(recommend(i)) == 1: #когда функция выдаёт 1 рекомендацию  и одно значение по умолчанию вместо Nan
            recommended_courses1.loc[i] = [recommend(i)[0][0][1], d[1]]
    else:
        recommended_courses1.loc[i] = [d[0], d[1]]  #когда функция выдаёт два значения по умолчанию

recommended_courses1

,Рекомендация № 1,Рекомендация № 2
490,566,551
357,571,1125
507,570,752
489,551,515
514,551,515
...,...,...
1198,551,566
1199,551,566
902,551,566
1160,551,566


In [16]:
#Проверим получившиеся данные на наличие пустых значений
b =  recommended_courses1.isnull().sum().sum()
b

0

In [17]:
#Создадим итоговую таблицу с основным курсом и двумя рекомендациями
recommended=recommended_courses1.reset_index().rename(columns={'index':'Основной курс'})
recommended

,Основной курс,Рекомендация № 1,Рекомендация № 2
0,490,566,551
1,357,571,1125
2,507,570,752
3,489,551,515
4,514,551,515
...,...,...,...
121,1198,551,566
122,1199,551,566
123,902,551,566
124,1160,551,566


In [18]:
#Сохраним получившийся датафрейм в Excel для отправки таблицы с рекомендациями продакт-менеджеру и в отдел маркетинга
for_printing = pd.ExcelWriter('Recomendations.xlsx')
recommended.to_excel(for_printing)
for_printing.save()


Итак, в итоговой таблице мы собрали основной курс с двумя рекомендациями. Задали минимальный порог вхождения, чтобы исключить рекомендации, которые встречаются слишком мало раз. И вместо таких малопопулярных курсов вывели два самых популярных в рекомендациях, тем самым желая увеличить средний чек в целом.

Если исключить рекомендацию самых популярных курсов вовсе, то это может отрицательно сказаться на продажах.
